In [1]:
import pandas as pd
import numpy as np
import os
if os.path.basename(os.getcwd()) == "Belfort":
    os.chdir('..')  ## to work at project root  like in any IDE
    os.chdir('..')

# General assumptions

In [2]:
r=0.04# discount rate
# Carbon tax
d_carbon_tax={2030:120,2040:170,2050:200,2060:200}

# 1. Technologies

In [3]:
Input_folder="Data/input/Prod_model/"
Technos_df=pd.read_csv(Input_folder+"Technos.csv",sep=";",decimal=",").set_index("TECHNOLOGIES")
Technos_df

,CAPEX_2030,CAPEX_2040,CAPEX_2050,CAPEX_2060,Decommissioning_2030,Decommissioning_2040,Decommissioning_2050,Decommissioning_2060,Lifetime_2030,Lifetime_2040,...,minCapacity_100_enr_2040,maxCapacity_100_enr_2040,minCapacity_100_enr_2050,maxCapacity_100_enr_2050,minCapacity_100_enr_2060,maxCapacity_100_enr_2060,RampConstraintPlus,RampConstraintMoins,RampConstraintPlus2,RampConstraintMoins2
TECHNOLOGIES,,,,,,,,,,,,,,,,,,,,,
CCG,830000,830000,830000,830000,0,0,0,0,30,30,...,0,12664,0,0,0,0,0.06,0.06,0,0
TAC,450000,450000,450000,450000,0,0,0,0,30,30,...,0,0,0,0,0,0,0.00,0.00,0,0
OldNuke,650000,650000,650000,650000,320000,320000,320000,320000,10,10,...,43880,43880,15520,15520,0,0,0.04,0.04,0,0
NewNuke,5000000,5000000,4500000,4500000,200000,200000,200000,200000,60,60,...,0,0,0,0,0,0,0.04,0.04,0,0
WindOnShore,1200000,1050000,900000,900000,0,0,0,0,25,30,...,30200,30200,37000,37000,37000,37000,0.00,0.00,0,0
WindOffShore,1700000,1500000,1300000,1300000,0,0,0,0,25,30,...,26200,78600,40000,120000,40000,120000,0.00,0.00,0,0
HydroRiver,0,0,0,0,0,0,0,0,75,75,...,11030,11030,11030,11030,11030,11030,0.00,0.00,0,0
NewHydroRiver,1800000,1800000,1800000,1800000,0,0,0,0,75,75,...,0,1200,0,1200,0,1200,0.00,0.00,0,0
Solar,945000,822000,742000,742000,0,0,0,0,25,30,...,70300,201300,100000,300000,100000,300000,0.00,0.00,0,0


In [4]:
def tech_tab(Technos_df,year,prod_hyp):
    df_ret=pd.DataFrame()
    df_ret['energyCost']=Technos_df['energyCost_'+str(year)]+Technos_df['CO2_emissions']*d_carbon_tax[year]/1000
    df_ret['capacityCost']=(Technos_df['CAPEX_'+str(year)]\
                            +Technos_df['Decommissioning_'+str(year)]/(1+r)**Technos_df['Lifetime_'+str(year)])\
                            *(1-1/(1+r))/(1-1/(1+r)**Technos_df['Lifetime_'+str(year)])\
                            +Technos_df['OM_'+str(year)]
    df_ret['minCapacity']=Technos_df['minCapacity_'+prod_hyp+'_'+str(year)]
    df_ret['maxCapacity']=Technos_df['maxCapacity_'+prod_hyp+'_'+str(year)]
    for col in ["EnergyNbhourCap","RampConstraintPlus","RampConstraintMoins","RampConstraintPlus2","RampConstraintMoins2"]:
        df_ret[col]=Technos_df[col]
    
    return df_ret

In [5]:
for year in [2030,2040,2050,2060]:
    for prod_hyp in ['nuclear_plus','nuclear_minus','100_enr']:
        df_tech=tech_tab(Technos_df,year,prod_hyp)
        df_tech.to_csv(Input_folder+"Technos_"+str(year)+"_"+prod_hyp+".csv",sep=";",decimal=",")

# 2. Storage

In [6]:
Input_folder="Data/input/Prod_model/"
StockTechnos_df=pd.read_csv(Input_folder+"Stock_technos.csv",sep=";",decimal=",").set_index("STOCK_TECHNO")
StockTechnos_df

,CAPEX_power_in_2030,CAPEX_power_out_2030,CAPEX_energy_2030,CAPEX_power_in_2040,CAPEX_power_out_2040,CAPEX_energy_2040,CAPEX_power_in_2050,CAPEX_power_out_2050,CAPEX_energy_2050,CAPEX_power_in_2060,...,efficiency_out_2030,efficiency_in_2040,dissipation_2040,efficiency_out_2040,efficiency_in_2050,dissipation_2050,efficiency_out_2050,efficiency_in_2060,dissipation_2060,efficiency_out_2060
STOCK_TECHNO,,,,,,,,,,,,,,,,,,,,,
Battery,1.705385e+05,0,225282.5768,156868.054,0,172728.3345,1.732686e+05,0,141662.1695,1.732686e+05,...,0.95,0.95,0.000000,0.95,0.95,0.000000,0.95,0.95,0.000000,0.95
HydroStorage,1.325000e+06,0,0.0000,1325000.000,0,0.0000,1.325000e+06,0,0.0000,1.325000e+06,...,0.90,0.90,0.000000,0.90,0.90,0.000000,0.90,0.90,0.000000,0.90
PowertoH2toPower,1.350000e+06,1100000,0.0000,1080000.000,1100000,0.0000,8.000000e+05,1100000,0.0000,8.000000e+05,...,0.50,0.70,0.000012,0.58,0.70,0.000012,0.58,0.70,0.000012,0.58


In [7]:
def stock_tech_tab(StockTechnos_df,year):
    df_ret=pd.DataFrame()
    for char in ['in','out','energy']:
        char1=char
        if char in ['in','out']:
            char1='power_'+char1
        df_ret['storageCost_'+char1]=StockTechnos_df['CAPEX_'+char1+'_'+str(year)]\
                            *(1-1/(1+r))/(1-1/(1+r)**StockTechnos_df['Lifetime_'+char])\
                            +StockTechnos_df['OPEX_'+char+'_'+str(year)]
    df_ret['p_max_in']=StockTechnos_df['p_max_in_'+str(year)]
    df_ret['p_max_out']=StockTechnos_df['p_max_out_'+str(year)]
    df_ret['efficiency_in']=StockTechnos_df['efficiency_in_'+str(year)]
    df_ret['dissipation']=StockTechnos_df['dissipation_'+str(year)]
    df_ret['efficiency_out']=StockTechnos_df['efficiency_out_'+str(year)]
    for col in ["p_max_out_eq_p_max_in","h_max"]:
        df_ret[col]=StockTechnos_df[col]
    
    return df_ret

In [8]:
for year in [2030,2040,2050,2060]:
    df_stock_tech=stock_tech_tab(StockTechnos_df,year)
    df_stock_tech.to_csv(Input_folder+"Stock_technos_"+str(year)+".csv",sep=";",decimal=",")

# 3. Hydrogen

In [9]:
#Input_folder="Data/input/Conso_model/H2/"
#H2Technos_df=pd.read_csv(Input_folder+"ParametersH2.csv",sep=";",decimal=",").set_index("H2_TECHNO")
#H2Technos_df

,CAPEX_2030,CAPEX_2040,CAPEX_2050,CAPEX_2060,Lifetime_2030,Lifetime_2040,Lifetime_2050,Lifetime_2060,OPEX_2030,OPEX_2040,OPEX_2050,OPEX_2060,energyCostH2,EfficiencyH2_2030,EfficiencyH2_2040,EfficiencyH2_2050,EfficiencyH2_2060
H2_TECHNO,,,,,,,,,,,,,,,,,
Electrolysis,1350000,1080000,800000,800000,15,15,15,15,54000,43200,32000,32000,0,0.650000,0.700000,0.700000,0.700000
CCH2,1100000,1100000,1100000,1100000,30,30,30,30,44000,44000,44000,44000,0,0.500000,0.580000,0.580000,0.580000
Storage,0,0,0,0,30,30,30,30,0,0,0,0,15,0.999988,0.999988,0.999988,0.999988


In [10]:
#def H2_tech_tab(H2Technos_df,year):
    #df_ret=pd.DataFrame()
    #df_ret['capacityCostH2']=H2Technos_df['CAPEX_'+str(year)]\
                            #*(1-1/(1+r))/(1-1/(1+r)**H2Technos_df['Lifetime_'+str(year)])\
                            #+H2Technos_df['OPEX_'+str(year)]
    #df_ret['energyCostH2']=H2Technos_df['energyCostH2']
    #df_ret['efficiencyH2']=H2Technos_df['EfficiencyH2_'+str(year)]
    
    #return df_ret

In [11]:
#for year in [2030,2040,2050,2060]:
    #df_H2_tech=H2_tech_tab(H2Technos_df,year)
    #df_H2_tech.to_csv(Input_folder+"H2_technos_"+str(year)+".csv",sep=";",decimal=",")

# 4. Flexibility

In [9]:
Input_folder="Data/input/Conso_model/Flex/"
FlexTechnos_df=pd.read_csv(Input_folder+"FlexParameters.csv",sep=";",decimal=",").set_index("FLEX_CONSUM")
FlexTechnos_df

,CAPEX_2030,CAPEX_2040,CAPEX_2050,CAPEX_2060,Lifetime_2030,Lifetime_2040,Lifetime_2050,Lifetime_2060,OPEX_2030,OPEX_2040,OPEX_2050,OPEX_2060,flex_ratio,max_ratio,flex_type,labourcost
FLEX_CONSUM,,,,,,,,,,,,,,,,
Steel,2.772006e+06,2.772006e+06,2.772006e+06,2.772006e+06,20,20,20,20,83160.18613,83160.18613,83160.18613,83160.18613,0.9,1,week,78
EV,3.907226e+05,3.441756e+05,3.441756e+05,3.441756e+05,15,15,15,15,22681.81818,22681.81818,22681.81818,22681.81818,0.5,1,day_ev,0


In [10]:
def Flex_tech_tab(FlexTechnos_df,year):
    df_ret=pd.DataFrame()
    df_ret['LoadCost']=FlexTechnos_df['CAPEX_'+str(year)]\
                            *(1-1/(1+r))/(1-1/(1+r)**FlexTechnos_df['Lifetime_'+str(year)])\
                            +FlexTechnos_df['OPEX_'+str(year)]
    for col in ["flex_ratio","max_ratio","flex_type","labourcost"]:
        df_ret[col]=FlexTechnos_df[col]
    
    return df_ret

In [11]:
for year in [2030,2040,2050,2060]:
    df_flex_tech=Flex_tech_tab(FlexTechnos_df,year)
    df_flex_tech.to_csv(Input_folder+"Flex_"+str(year)+".csv",sep=";",decimal=",")